#### **NEM Review contract co-design workshop**

# Contract performance modelling: simulations

## Prepare environment and data

In [49]:
# Data handling
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import pyarrow.compute as pc

# Visualisation
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Custom functions
from scripts.common_functions import python_setup, get_mms_data, save_figure

# Set up Python
working_dir, charts_dir, data_dir = python_setup(2)

# Paths and variables
gen_info_path = os.path.join(data_dir, "gen_info.csv")
duid_data_path = os.path.join(data_dir, "duid_data.parquet")
market_data_path = os.path.join(data_dir, "market_data.parquet")
regions = ["NSW", "QLD", "SA", "VIC"]

# Load gen info
gen_info = pd.read_csv(
    gen_info_path,
    index_col="DUID")

## Finance and cost assumptions

Financing assumptions are variables, which can be adjusted. Cost assumptions are sourced from [CSIRO's GenCost 2024-25](https://www.csiro.au/en/research/technology-space/energy/Electricity-transition/GenCost) (using 2024 prices under the 'current policies' scenario).

#### Finance

| Metric | Value |
|-|-|
| Leverage ratio | 65% |
| Loan rate | 5% |
| Equity rate | 15% |
| Weighted average cost of capital | 8.5% |

#### Costs

| | Wind | Solar |
|-|-|-|
| Capital expenditure *(\$/kW)* | $3,351 | $1,463 |
| Economic life *(years)* | 25 | 30 |
| Fixed operating & maintenance costs *(\$/kW/year)* | $28 | $12 |
| Annualised costs ($/MW/year) | **$355,432** | **$148,133** |

In [50]:
# Costs and finance variables
costs = {
    "Wind": {
        "Capex": 3351,
        "Economic life": 25,
        "FOM": 28},
    "Solar": {
        "Capex": 1463,
        "Economic life": 30,
        "FOM": 12,},
    "Financing": {
        "Loan": 0.65,
        "Equity": 0.35,
        "Loan rate": 0.05,
        "Equity rate": 0.15,
        "WACC": 0.085}}

for tech in ["Wind", "Solar"]:    
    capex = costs[tech]["Capex"]
    life = costs[tech]["Economic life"]
    fom = costs[tech]["FOM"]
    wacc = costs["Financing"]["WACC"]
    crf = wacc * (1 + wacc) ** life / ((1 + wacc) ** life - 1)
    annual_capex = capex * crf
    total_annual_cost_mw = (annual_capex + fom) * 1000
    costs[tech]["Annualised costs per MW"] = total_annual_cost_mw

costs

{'Wind': {'Capex': 3351,
  'Economic life': 25,
  'FOM': 28,
  'Annualised costs per MW': 355431.8480475924},
 'Solar': {'Capex': 1463,
  'Economic life': 30,
  'FOM': 12,
  'Annualised costs per MW': 148132.99168038458},
 'Financing': {'Loan': 0.65,
  'Equity': 0.35,
  'Loan rate': 0.05,
  'Equity rate': 0.15,
  'WACC': 0.085}}

In [60]:
# Wind DUIDs in SA
sa_wind_duids = gen_info[
    (gen_info["Region"] == "SA") &
    (gen_info["Technology"] == "Wind") &
    (gen_info["Late start"] == False)].index.tolist()

# Load CFs
simulated_stations = pd.DataFrame(
    index=sa_wind_duids)
simulated_stations.index.name = "DUID"
simulated_stations["Name"] = gen_info.loc[sa_wind_duids, "Name"]
simulated_stations["Maximum capacity"] = 250
simulated_stations["Capacity factor"] = gen_info.loc[sa_wind_duids, "Capacity factor"]
simulated_stations.sort_values("Capacity factor", ascending=False, inplace=True)
simulated_stations

,Name,Maximum capacity,Capacity factor
DUID,,,
HDWF1,Hornsdale Wind Farm Stage 1,250,0.393370
SNOWSTH1,Snowtown S2 Wind Farm,250,0.372119
HDWF3,Hornsdale Wind Farm Stage 3,250,0.370498
HDWF2,Hornsdale Wind Farm Stage 2,250,0.370263
LGAPWF1,Lincoln Gap Wind Farm - stage 1,250,0.359292
SNOWTWN1,Snowtown Wind Farm,250,0.347804
NBHWF1,Hallett 4 North Brown Hill,250,0.336183
WGWF1,Willogoleche Wind Farm,250,0.335794
HALLWF2,Hallett Stage 2 Hallett Hill,250,0.332435
